In [46]:
# run on python 3.9 or lesser - as sentence piece throws an error while installing frame semantic transformer
! pip install frame_semantic_transformer

import nltk
import ssl 
from frame_semantic_transformer import FrameSemanticTransformer

In [28]:
import pandas as pd 
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download('wordnet')
nltk.download('framenet_v17')

[nltk_data] Downloading package wordnet to /Users/avtk/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package framenet_v17 to
[nltk_data]     /Users/avtk/nltk_data...
[nltk_data]   Package framenet_v17 is already up-to-date!


True

In [98]:
import pickle
import logging 

logger = logging.getLogger()

def chunks(df: pd.DataFrame, chunk_size: int):
    """Function to yield chunks

    Args:
        df (pd.DataFrame): Dataframe for region
        chunk_size (int): size of chunks

    Yields:
        df: Dataframe of chunk size
    """
    for i in range(0, len(df), chunk_size):
        yield df.iloc[i:i + chunk_size]

def get_frame_semantics(filename: str, region: str):
    """Function to get frame semantics 

    Args:
        df (pd.DataFrame): dataframe for region
        filename (str): name of file
    """
    
    df = pd.read_csv(filename)
    frame_transformer = FrameSemanticTransformer(batch_size=16) 
    
    iterator = chunks(df, 16)
    pickle_obj = {"indices": [], "frame_semantics": []} 
    for chunk in iterator:
        print(chunk)
        try:
            result = frame_transformer.detect_frames_bulk(chunk.title.values)
        except Exception as e: 
            logger.error(e)
        else: 
            indices = list(chunk.index)
            pickle_obj["indices"].extend(indices)
            pickle_obj["frame_semantics"].extend(result) 
            pickle.dump(pickle_obj, file = open(f"./data/processed/frame_semantics_{region}.pickle", "wb"))
            break

region = ""
get_frame_semantics(f"./data/raw/{region}.csv")
    

                                                   id         indexed_date  \
0   a19ca4072e72b22cf0c42fc821233416b9ebd0118c3eae...  2024-02-01 00:00:00   
1   408e882043e5bf2a93e2002f77c365cbe724e9e67239af...  2024-02-01 00:00:00   
2   1c8e6ea9bc4d35a92304d7a354441fd0cca80b17040119...  2024-02-01 00:00:00   
3   e9d4ba5383da4158ca767fd8b171a0653b7632acad3eaf...  2024-02-01 00:00:00   
4   e53ded3b4223329b447fd3cddb3d7e6d2b120f92f199e4...  2024-02-01 00:00:00   
5   db10ed71d127b4833069a24ce2afaca7db6d44fbfa8edf...  2024-02-01 00:00:00   
6   78d5385ef5d672981cb074e518a0a819f98dd82e1a77da...  2024-02-01 00:00:00   
7   099097df65d5e33efa7dee3f1620dfe0787b837aae2e38...  2024-01-31 00:00:00   
8   5c154ff870b7bc69c2203aed636210b4bc056dd16b64c7...  2024-02-01 00:00:00   
9   c25af40d2d1f5f980f92e806190f38b737f7a24c7f5875...  2024-02-01 00:00:00   
10  61b1ba172ac1266e2bd03ec17e7df48be70bbe025b28ca...  2024-02-01 00:00:00   
11  1f7e89f90a2bd8142ea454308c2dd46d2a75b0ed6d1c6c...  2024-02-0

['Pentagon warns ‘there will be consequences’ for strikes that killed US soldiers'
 'Where Joe Biden could target with strikes after three US troops killed in Jordan'
 "Abandoning Ukraine now would be 'an own goal of historic proportions', CIA ch..."
 'Israeli forces kill three Palestinian fighters in West Bank hospital raid'
 'US names three soldiers killed by Jordan drone ‘mistaken for American aircraft’'
 "At least half of Gaza's buildings damaged or destroyed, new analysis shows"
 'Biden says he has DECIDED how U.S. will retaliate against Iran-backed drone strike that killed three U.S. troops in Jordan'
 'Pentagon mulls response to Jordan drone strike amid fears for Gaza ceasefire efforts'
 "Heartbroken family of US soldier, 24, killed in Jordan strike reveal she died alongside her best friend, 23, after following in her grandfather and father's footsteps to join the military"
 'Israel-Gaza war: Counting the destruction of religious sites'
 'Iran-backed militia halts attacks on US 